In [1]:
%load_ext autoreload
import os

os.chdir(os.path.abspath(os.curdir).replace('notebooks',''))

In [2]:

from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import pandas as pd
import json


In [3]:
import os
with open('openai.key','r') as f:
    openai_api_key = f.read()
os.environ['OPENAI_API_KEY'] = openai_api_key 

In [23]:
# Temp = 0 so that we get clean information without a lot of creativity
chat_model = ChatOpenAI(temperature=0.9, openai_api_key=openai_api_key)

In [35]:


print (output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"character_name": string  // This is the name of the character
	"character_age": string  // This is the age of the character
	"character_gender": string  // Character gender: male, female, non-binary
	"character_traits": string  // This briefly describes the character profile, their background, traits, habits and goals
	"character_role": string  // This is the role they play in the story: are they central or secondary, helping the protagonist, ignoring, thwarting or deceiving them, do they grow and how
}
```


In [36]:
author_name = 'H. P. Lovecraft'

author_style = 'described as atmospheric, dark, and highly descriptive. His works often revolve around cosmic horror and the unknown, incorporating elements of science fiction and supernatural themes. Lovecraft\'s prose is characterized by intricate world-building, richly detailed descriptions of otherworldly creatures and settings, and a sense of foreboding and dread that pervades his stories. He frequently employs a first-person narrative, immersing readers in the unsettling perspectives of his characters as they confront the unfathomable horrors of the Lovecraftian universe.'

story_summary = 'A small coastal town is plagued by a series of strange disappearances. As the townsfolk investigate, they uncover a cult worshipping an ancient entity that dwells in the depths of the nearby forest. The cult seeks to awaken the deity, intending to bring about the end of the world. The townspeople must unravel the secrets of the cult before it\'s too late.'

In [113]:
story_summary

"A small coastal town is plagued by a series of strange disappearances. As the townsfolk investigate, they uncover a cult worshipping an ancient entity that dwells in the depths of the nearby forest. The cult seeks to awaken the deity, intending to bring about the end of the world. The townspeople must unravel the secrets of the cult before it's too late."

In [111]:
data_storage = './data/stories'
existing_story = 'lovecraft0'

path_summary = os.path.join(data_storage,existing_story,'00_story_summary.txt')

with open(path_summary,'w') as f:
    f.write(story_summary)

In [112]:
with open(path_summary,'r') as f:
    story_summary = f.read()

In [38]:
output = chat_model(_input.to_messages())

if "```json" in output.content:
    json_string = output.content.split("```json")[1].replace('```','').strip()
else:
    json_string = output.content

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


In [28]:


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

There are 1 message(s)
Type: <class 'langchain.schema.HumanMessage'>
---------------------------

You are a storyteller who writes and is inspired by H. P. Lovecraft and you write stories in their style. 
Their style is typically described as described as atmospheric, dark, and highly descriptive. His works often revolve around cosmic horror and the unknown, incorporating elements of science fiction and supernatural themes. Lovecraft's prose is characterized by intricate world-building, richly detailed descriptions of otherworldly creatures and settings, and a sense of foreboding and dread that pervades his stories. He frequently employs a first-person narrative, immersing readers in the unsettling perspectives of his characters as they confront the unfathomable horrors of the Lovecraftian universe.. You are writing a story about A small coastal town is plagued by a series of strange disappearances. As the townsfolk investigate, they uncover a cult worshipping an ancient entity that dw

In [29]:
output = chat_model(_input.to_messages())

if "```json" in output.content:
    json_string = output.content.split("```json")[1].replace('```','').strip()
else:
    json_string = output.content


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


In [40]:
characters_json = json.loads(json_string)



In [58]:
def character_builder(char_dict):
    return f'Name: {char_dict["character_name"]} ({char_dict["character_age"]}), {char_dict["character_gender"]}' \
        f' Role: {char_dict["character_role"]}, Bio: {char_dict["character_traits"]}\n'




In [62]:
character_descriptions = '\n'.join([character_builder(js) for js in characters_json])

print(len(character_descriptions.split()))
print(character_descriptions)

317
Name: Adam Cross (45), male Role: central character, helping the protagonist, Bio: Adam is the local Sheriff of the small coastal town. He's hardworking, determined, and highly protective of the people in his town. He's been investigating the strange disappearances and is determined to get to the bottom of it.

Name: Alice Chen (28), female Role: central character, helping the protagonist, Bio: Alice is a curious journalist investigating the strange happenings in the town. She is brave and headstrong, sometimes recklessly so. She's determined to uncover the truth, even if it means putting herself in danger.

Name: Josephine Blake (60), female Role: secondary character, helping the protagonist, Bio: Josephine is the oldest resident of the town and the owner of the local bookstore. She's well-read and knowledgeable about the occult. She's a bit eccentric and has a strange fascination with the strange happenings in the town.

Name: Ethan Davis (32), male Role: secondary character, hel

In [60]:
# How you would like your reponse structured. This is basically a fancy prompt template

print (output_parser.get_format_instructions())

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"section_name": string  // This is the name of the section of the story
	"section_description": string  // This is a short one sentence description of what happens in this section of the story
	"characters": string  // This is a comma separated list of the characters who take part in each section of the story
}
```


In [63]:


_input = prompt.format_prompt(
    author_name = author_name, 
    author_style = author_style,
    story_summary = story_summary,
    character_descriptions = character_descriptions
)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

There are 1 message(s)
Type: <class 'langchain.schema.HumanMessage'>
---------------------------

You are a storyteller who writes and is inspired by H. P. Lovecraft and you write stories in their style. 
Their style is typically described as described as atmospheric, dark, and highly descriptive. His works often revolve around cosmic horror and the unknown, incorporating elements of science fiction and supernatural themes. Lovecraft's prose is characterized by intricate world-building, richly detailed descriptions of otherworldly creatures and settings, and a sense of foreboding and dread that pervades his stories. He frequently employs a first-person narrative, immersing readers in the unsettling perspectives of his characters as they confront the unfathomable horrors of the Lovecraftian universe.. 

This is the story summary: 
A small coastal town is plagued by a series of strange disappearances. As the townsfolk investigate, they uncover a cult worshipping an ancient entity that dw

In [64]:
output = chat_model(_input.to_messages())

if "```json" in output.content:
    json_string = output.content.split("```json")[1].replace('```','').strip()
else:
    json_string = output.content

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


In [65]:
characters_json = json.loads(json_string)

characters_json

[{'section_name': 'Introduction',
  'section_description': 'The town is introduced along with the disappearances',
  'characters': 'Adam Cross, Alice Chen'},
 {'section_name': 'The First Clues',
  'section_description': 'The protagonists start their investigation and meet Josephine',
  'characters': 'Adam Cross, Alice Chen, Josephine Blake'},
 {'section_name': "The Fisherman's Tales",
  'section_description': 'Ethan shares his knowledge of strange happenings at sea',
  'characters': 'Adam Cross, Ethan Davis'},
 {'section_name': 'Uncovering the Cult',
  'section_description': 'The protagonists discover the cult and their intentions',
  'characters': 'Adam Cross, Alice Chen, Josephine Blake, Nate Holt, Evelyn Kingston'},
 {'section_name': 'Preparations and Confrontations',
  'section_description': "The protagonists prepare to stop the cult's plans and face their biggest challenges",
  'characters': 'Adam Cross, Alice Chen, Josephine Blake, Ethan Davis, Nate Holt, Evelyn Kingston'},
 {'se

In [110]:
characters_json

[{'section_name': 'Introduction',
  'section_description': 'The town is introduced along with the disappearances',
  'characters': 'Adam Cross, Alice Chen'},
 {'section_name': 'The First Clues',
  'section_description': 'The protagonists start their investigation and meet Josephine',
  'characters': 'Adam Cross, Alice Chen, Josephine Blake'},
 {'section_name': "The Fisherman's Tales",
  'section_description': 'Ethan shares his knowledge of strange happenings at sea',
  'characters': 'Adam Cross, Ethan Davis'},
 {'section_name': 'Uncovering the Cult',
  'section_description': 'The protagonists discover the cult and their intentions',
  'characters': 'Adam Cross, Alice Chen, Josephine Blake, Nate Holt, Evelyn Kingston'},
 {'section_name': 'Preparations and Confrontations',
  'section_description': "The protagonists prepare to stop the cult's plans and face their biggest challenges",
  'characters': 'Adam Cross, Alice Chen, Josephine Blake, Ethan Davis, Nate Holt, Evelyn Kingston'},
 {'se

In [72]:
i = 0

section_name = characters_json[i]['section_name']
section_description = characters_json[i]['section_description']
mentioned_characters = characters_json[i]['characters']

In [73]:

_input = prompt.format_prompt(
    author_name = author_name, 
    author_style = author_style,
    story_summary = story_summary,
    character_descriptions = character_descriptions,
    section_name = section_name,
    section_description = section_description,
    mentioned_characters = mentioned_characters
    
)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

There are 1 message(s)
Type: <class 'langchain.schema.HumanMessage'>
---------------------------

You are a storyteller who writes and is inspired by H. P. Lovecraft and you write stories in their style. 
Their style is typically described as described as atmospheric, dark, and highly descriptive. His works often revolve around cosmic horror and the unknown, incorporating elements of science fiction and supernatural themes. Lovecraft's prose is characterized by intricate world-building, richly detailed descriptions of otherworldly creatures and settings, and a sense of foreboding and dread that pervades his stories. He frequently employs a first-person narrative, immersing readers in the unsettling perspectives of his characters as they confront the unfathomable horrors of the Lovecraftian universe.. 

This is the story summary: 
A small coastal town is plagued by a series of strange disappearances. As the townsfolk investigate, they uncover a cult worshipping an ancient entity that dw

In [75]:
output = chat_model(_input.to_messages())

if "```json" in output.content:
    json_string = output.content.split("```json")[1].replace('```','').strip()
else:
    json_string = output.content


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


In [104]:
section_list = ['The small coastal town of Kingsport was a picturesque and idyllic place, nestled on the edge of the ocean and surrounded by lush forests. But beneath the surface, something sinister was brewing. Over the past few weeks, a string of strange disappearances had rocked the town, leaving residents shaken and fearful.', 'Sheriff Adam Cross had been working tirelessly to investigate the disappearances, but with no clues or leads, he was at a loss. That was until Alice Chen, a young and curious journalist, arrived in town asking questions and stirring up trouble.', 'As Adam and Alice began to uncover the truth behind the disappearances, they realized that something much larger and more dangerous was at play. A cult worshipping an ancient entity had taken root in the forest, and they were seeking to awaken it, intending to bring about the end of the world.', 'Adam and Alice knew they had to act fast to stop the cult and save the town from destruction. But as they delved deeper into the darkness, they realized that the lines between good and evil were not as clear-cut as they had once thought. The horrors they were about to face would push them to their limits and threaten to consume them both.']


In [116]:
story_so_far = section_list[0]
this_scene = [f"Scene {i}: {s}\n\n" for i, s in enumerate(section_list[1:])]

this_scene = ''.join(this_scene)

In [130]:
json.loads(json_string)

[{'scene_content': "NARRATOR: Sheriff Adam Cross was pacing back and forth in his office, frustration etched on his face. \nADAM CROSS: (sighs) I don't know what else to do. I've searched the town, talked to witnesses, looked into every possible lead and still nothing. \nJust then, there was a knock on the door. \nADAM CROSS: Come in. \nAlice Chen walked in, an air of confidence about her. \nALICE CHEN: Sheriff Cross, I'm Alice Chen from the Arkham Gazette. I'm here investigating the disappearances in your town. \nADAM CROSS: (skeptical) And how do I know I can trust you? \nALICE CHEN: You don't. But I can tell you that I'm not leaving until I get the answers I need. And maybe I can help you in the process. \nADAM CROSS: (pauses) Alright. But be careful. This town is not what it seems. \nNARRATOR: With that, a bond was formed between the Sheriff and the journalist, two people determined to uncover the truth.",
  'scene_image': 'An image of Adam Cross pacing in his office while Alice Ch

In [137]:
import yaml
yaml.dump({'a':23,'b':334},open('./ad.yaml','w'), )

In [119]:
from src.prompts.storyteller_prompts import scene_prompt

_input = scene_prompt.format_prompt(
    author_name = author_name, 
    author_style = author_style,
    story_summary = story_summary,
    character_descriptions = character_descriptions,
    section_name = section_name,
    section_description = section_description,
    mentioned_characters = mentioned_characters,
    story_so_far = story_so_far,
    this_scene = this_scene
)


print (f"There are {len(_input.messages)} message(s)")
print (f"Type: {type(_input.messages[0])}")
print ("---------------------------")
print (_input.messages[0].content)

There are 1 message(s)
Type: <class 'langchain.schema.HumanMessage'>
---------------------------

You are a scriptwriter who writes and is inspired by H. P. Lovecraft and you write stories in their style. 
Their style is typically described as described as atmospheric, dark, and highly descriptive. His works often revolve around cosmic horror and the unknown, incorporating elements of science fiction and supernatural themes. Lovecraft's prose is characterized by intricate world-building, richly detailed descriptions of otherworldly creatures and settings, and a sense of foreboding and dread that pervades his stories. He frequently employs a first-person narrative, immersing readers in the unsettling perspectives of his characters as they confront the unfathomable horrors of the Lovecraftian universe.. 

This is the story summary: 
A small coastal town is plagued by a series of strange disappearances. As the townsfolk investigate, they uncover a cult worshipping an ancient entity that d

In [120]:
output = chat_model(_input.to_messages())

if "```json" in output.content:
    scene_detail = output.content.split("```json")[1].replace('```','').strip()
else:
    scene_detail = output.content


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


In [123]:
json.loads(scene_detail)

[{'scene_content': "NARRATOR: Sheriff Adam Cross had been working tirelessly to investigate the disappearances, but with no clues or leads, he was at a loss. That was until Alice Chen, a young and curious journalist, arrived in town asking questions and stirring up trouble.\n\nADAM CROSS: Can I help you with something?\n\nALICE CHEN: Yes, I'm here to investigate the strange happenings in this town. I've heard there have been some unexplained disappearances.\n\nADAM CROSS: That's right, we're still trying to get to the bottom of it. Do you have any leads?\n\nALICE CHEN: Not yet, but I plan on turning over every stone until I find the truth.\n\nNARRATOR: Adam had mixed feelings about this young journalist, but he couldn't deny that she had the drive and determination to get to the bottom of things. The two of them would soon team up to uncover the dark secrets of Kingsport.",
  'scene_image': "A dimly lit police station, with Adam and Alice standing opposite each other. There's a map of 

In [126]:
json.loads(json_string)[0]['scene_image']

[print(j['scene_content']+'\n\n') for j in json.loads(scene_detail)]

NARRATOR: Sheriff Adam Cross had been working tirelessly to investigate the disappearances, but with no clues or leads, he was at a loss. That was until Alice Chen, a young and curious journalist, arrived in town asking questions and stirring up trouble.

ADAM CROSS: Can I help you with something?

ALICE CHEN: Yes, I'm here to investigate the strange happenings in this town. I've heard there have been some unexplained disappearances.

ADAM CROSS: That's right, we're still trying to get to the bottom of it. Do you have any leads?

ALICE CHEN: Not yet, but I plan on turning over every stone until I find the truth.

NARRATOR: Adam had mixed feelings about this young journalist, but he couldn't deny that she had the drive and determination to get to the bottom of things. The two of them would soon team up to uncover the dark secrets of Kingsport.


NARRATOR: As Adam and Alice began to uncover the truth behind the disappearances, they realized that something much larger and more dangerous w

[None, None, None]

In [127]:
[print(j['scene_image']+'\n\n') for j in json.loads(scene_detail)]

A dimly lit police station, with Adam and Alice standing opposite each other. There's a map of the town on the wall behind them.


A dark and foreboding forest, with twisted trees and tangled underbrush. In the distance, the faint glow of torches can be seen.


A dimly lit clearing in the woods, with a group of robed cultists gathered around a central altar. The altar is adorned with strange symbols and glowing runes.




[None, None, None]

In [138]:
from story_teller import StoryTeller

lovecraft0 = StoryTeller(
    author_name = 'H. P. Lovecraft',
    author_style = 'described as atmospheric, dark, and highly descriptive. His works often revolve around cosmic horror and the unknown, incorporating elements of science fiction and supernatural themes. Lovecraft\'s prose is characterized by intricate world-building, richly detailed descriptions of otherworldly creatures and settings, and a sense of foreboding and dread that pervades his stories. He frequently employs a first-person narrative, immersing readers in the unsettling perspectives of his characters as they confront the unfathomable horrors of the Lovecraftian universe.',
    data_storage = './data/stories',
    story_key = 'lovecraft0'
)
lovecraft0.create_story_summary()
lovecraft0.create_story_characters()
lovecraft0.create_story_plot()

In [139]:
lovecraft0.story_plot

[{'characters': 'Evelyn Kline, Thomas Marshall',
  'section_description': 'Evelyn arrives in the small coastal town to investigate the disappearances.',
  'section_name': 'Arrival'},
 {'characters': 'Evelyn Kline, Thomas Marshall',
  'section_description': "Evelyn and Thomas investigate the disappearances and start to unravel the cult's secrets.",
  'section_name': 'The Investigation'},
 {'characters': 'Evelyn Kline, Thomas Marshall, Jacob Grey, Maria Hernandez',
  'section_description': 'The townspeople discover the existence of the cult and their plans to awaken an ancient deity.',
  'section_name': 'The Cult'},
 {'characters': 'Evelyn Kline, Thomas Marshall, Jacob Grey, Maria Hernandez',
  'section_description': "Evelyn discovers the truth behind the disappearances and the cult's plan to end the world.",
  'section_name': 'The Revelation'},
 {'characters': 'Evelyn Kline, Thomas Marshall, Jacob Grey, Maria Hernandez',
  'section_description': 'Evelyn and Thomas confront Jacob and the

In [140]:
import requests

e11labs_key = open('./tts/elevenlabs.key','r').read()

In [141]:
e11labs_key

'9091911747032ad2aaf48947c11a1456'

In [207]:
txt = open('./data/stories/lovecraft0/03_section_detail_2_prompt.txt','r').readlines()

txt[0].split('\n')

['You are a storyteller who writes and is inspired by H. P. Lovecraft and you write stories in their style. Their style is typically described as described as atmospheric, dark, and highly descriptive. His works often revolve around cosmic horror and the unknown, incorporating elements of science fiction and supernatural themes. Lovecraft\'s prose is characterized by intricate world-building, richly detailed descriptions of otherworldly creatures and settings, and a sense of foreboding and dread that pervades his stories. He frequently employs a first-person narrative, immersing readers in the unsettling perspectives of his characters as they confront the unfathomable horrors of the Lovecraftian universe.. This is the story summary: A small coastal town is plagued by a series of strange disappearances. As the townsfolk investigate, they uncover a cult worshipping an ancient entity that dwells in the depths of the nearby forest. The cult seeks to awaken the deity, intending to bring abo

In [213]:
txt = 'Bob\nBob'
with open('./bob.txt','w') as f:
    f.writelines(txt)

In [211]:
txt = 'Bob\nBob'
with open('./bob.txt','w') as f:
    f.write(txt)

In [143]:
from elevenlabs import set_api_key
set_api_key(e11labs_key)

In [149]:
from elevenlabs import generate, play, set_api_key, voices

# set_api_key("<YOUR_API_KEY>") # Check the guide on how to get a free API key: https://docs.elevenlabs.io/authentication/01-xi-api-key 


In [ ]:
generate()

In [148]:
file_path = './hi.mp3'

with open(file_path, "wb") as out:
    # Write the response to the output file.
    out.write(audio)

In [155]:
import json

[j['name'] for j in json.load(open('./tts/voices.json','r'))['voices']]

['Rachel', 'Domi', 'Bella', 'Antoni', 'Elli', 'Josh', 'Arnold', 'Adam', 'Sam']

In [158]:
characters = yaml.load(open('./data/stories/lovecraft0/01_characters.yaml','r'), Loader=yaml.FullLoader)

In [225]:
from src.voice_selection import voice_assignment

In [226]:
voice_assignments = voice_assignment(characters)

In [227]:
voice_assignments

{'NARRATOR': 'Antoni',
 'THOMAS MARSHALL': 'Arnold',
 'EVELYN KLINE': 'Domi',
 'JACOB GREY': 'Adam',
 'MARIA HERNANDEZ': 'Rachel'}

In [169]:
characters_simple = [{
    'character_name': c['character_name'],
    'character_age': c['character_age'],
    'character_gender': c['character_gender']
} for c in characters]

characters_simple


[{'character_name': 'Thomas Marshall',
  'character_age': '55',
  'character_gender': 'male'},
 {'character_name': 'Evelyn Kline',
  'character_age': '30',
  'character_gender': 'female'},
 {'character_name': 'Jacob Grey',
  'character_age': '40',
  'character_gender': 'male'},
 {'character_name': 'Maria Hernandez',
  'character_age': '25',
  'character_gender': 'female'}]

In [251]:
json.load(open('./data/stories/lovecraft0/04_1_section_plot_safe.txt','r').read())

FileNotFoundError: [Errno 2] No such file or directory: './data/stories/lovecraft0/04_1_section_plot_safe.txt'

In [235]:
scenes = yaml.load(open('./data/stories/lovecraft0/04_2_section_plot.yaml','r'), Loader=yaml.FullLoader)

In [244]:

for scene in scenes:
    for s in scene['scene_content'].split('\n'):
        print(s)
    print('')

NARRATOR: As Evelyn and Thomas continue to investigate the strange disappearances in the small coastal town, they stumble upon a group of people meeting in an abandoned warehouse on the outskirts of town. They watch from the shadows as the group, dressed in hooded robes and chanting in an unfamiliar language, is led by a man with a shaved head and a bushy beard, named Jacob Grey. His followers listen intently as he speaks in a commanding voice.

NARRATOR: The meeting comes to an end and the cult members begin to disperse, but Evelyn and Thomas follow one of the cult members, Maria Hernandez, into an alleyway. They confront her, demanding answers.

EVELYN KLINE: What's going on here, Maria? What is this cult? 
MARIA HERNANDEZ: I-I can't say... You don't understand. If they find out I'm talking to you... 
THOMAS MARSHALL: We're not going to hurt you, Maria. We just want to know what's happening in this town. 
MARIA HERNANDEZ: (sighs) Fine. But you didn't hear this from me. They believe i

In [252]:
text = 'NARRATOR: This is it\nThomas: I am tired'


In [219]:
text_split

['NARRATOR: THIS IS IT', 'THOMAS: I AM TIRED']

In [222]:
voice

''

In [229]:
voice_assignments

{'NARRATOR': 'Antoni',
 'THOMAS MARSHALL': 'Arnold',
 'EVELYN KLINE': 'Domi',
 'JACOB GREY': 'Adam',
 'MARIA HERNANDEZ': 'Rachel'}

In [253]:
len(str(0))

1

In [257]:
voice_script = yaml.load(open('./data/stories/lovecraft0/05_00_voice_scripts.yaml','r'), Loader=yaml.FullLoader)

In [261]:
len(' '.join([v['text'] for v in voice_script]))

1971

In [264]:
voice_script = yaml.load(open('./data/stories/lovecraft0/05_01_voice_scripts.yaml','r'), Loader=yaml.FullLoader)

len(' '.join([v['text'] for v in voice_script]))

1634

In [298]:
txt = open('./data/stories/donaldson0/04_0_section_plot_safe.txt','r').read()
txt = txt.replace('\\n', '').replace('\\', '').replace('\n', '').replace('  ', '').replace(',]', ']')
txt.strip()

'"[{"SCENE_CONTENT": ["NARRATOR: It was a beautiful day in the forest, and Sammy the Squirrel was out for his usual adventure.","NARRATOR: As he was scampering through the trees, he noticed something shiny on the ground. Curious, he ran over to investigate and found a small, mysterious key.","SAMMY THE SQUIRREL: What could this be for?","SAMMY THE SQUIRREL: (Wondering aloud) How old is this key? How long has it been here?","SAMMY THE SQUIRREL: (Holding the key up to the sunlight) This key must have a story behind it. I\'m determined to uncover its secrets!"],"SCENE_IMAGE": "A stunning vista of winding forest trails and flickering sunlight through the leaves. Sammy is seen holding up the key, looking at it closely.","SCENE_AUDIO": "A whimsical and upbeat melody begins to play, featuring woodwinds, strings, and a gentle percussion. The music reflects the wonder and curiosity of Sammy\'s discovery and excitement for the journey ahead."},{"SCENE_CONTENT": ["NARRATOR: Sammy knew that he cou

In [301]:
json.loads(txt.strip()[1:-1])

SyntaxError: invalid syntax (<string>, line 1)

In [297]:
txt.strip()[1:-1][560:]

',],"SCENE_IMAGE": "A stunning vista of winding forest trails and flickering sunlight through the leaves. Sammy is seen holding up the key, looking at it closely.","SCENE_AUDIO": "A whimsical and upbeat melody begins to play, featuring woodwinds, strings, and a gentle percussion. The music reflects the wonder and curiosity of Sammy\'s discovery and excitement for the journey ahead."},{"SCENE_CONTENT": ["NARRATOR: Sammy knew that he couldn\'t solve this mystery alone.","NARRATOR: Without wasting another moment, he set off on his new adventure. He bounded through the forest, his heart racing with excitement.","NARRATOR: He knew that he would need the help of his good friends.","SAMMY THE SQUIRREL: (shouting) Benny, Benny!","BENNY THE BUNNY: (hopping over) What\'s happening Sammy, what did you find?","SAMMY THE SQUIRREL: (Holding up the key) It\'s a key!","BENNY THE BUNNY: (in amazement) Wow! What do you think it opens?","SAMMY THE SQUIRREL: I don\'t know, but I\'m going to find out! Will

In [ ]:
[pnp]